In [0]:
#### Coalesce(numPartitions): Decrease number of partitions in the RDD to numPartitions.
Useful for running operations more efficiently after filtering down a large dataset.
It just combine the existing partition to reduce the number of partitions 
# repartition(numPartitions): reshuffle the data in the RDD randomly to create either more or fewer partitions and balance it across them.
this always shuffles all data over the network.
key ideas:
1. operations which can cause a shuffle iclude repartitions like repartition and coalesce.
2. Repartition(1) and Coalesce(1) can be used to output data in a shuffle  file but Coalesce(1) will perform better as no reshuffle occurs.
3. number of partition =# of CPU*4
4. When some empty or light files are produced after the final processing, use coalesce.
->>> Coalesce is preferred over reparation if we have to reduce the count of partitions in an RDD
->>> If data is skewness, then repartition is preferred but data is not skewed then repartition is preferred
->>> As a thumb, each output file should be around 128MB or 256MB,
if that is not the case, use coalesce in the stage of the pipeline where partitions in the output have much lesser or o data at all.
->>> If we will perform some filter operations or other operations that 
will trigger empty partition then at the time coalesce is called better 
->>> Avoiding shuffles: Shuffle are quite expensive, when we have joined
  a bigger dataset quite frequently with some smaller dataset which are 
  coming at a higher speed and time frame then, we can cache the bigger RDD
  and whenever a new data comes in you join with (when joining a bigger dataset
  frequently with smaller datasets on the same key, look for opportunity to avoid shuffle) with the cached RDD, 
  when we cached the RDD in the memory, then reading the file again and again, you do not avoid shuffle. 
  You cache the RDD and you repartition the RDD as well based on the field over which you would like to join, 
  so we can think like pair RDD, where we can construct the pair RDD with key as the field with which 
  you are going to join and you cache the bigger RDD and re-partition it as well when we do that the smaller
  files that keeps on arriving and they have to be constantly joined with the bigger data sets only the smaller
  data set and has to be shuffled and the bigger dataset will remain static. 
  Shuffling on the smaller dataset is faster as compared to shuffling both the data sets 
->>> If we have some kind of topN, rank or row number operations, prefer secondary sorting.
->>> Prefer reduceByKey or combineByKey over groupByKey as the former reduces the amount of data that gets shuffled.
->>> Avoid collecting larger results in memory – the output of the spark application is not 
required to be displayed to the user in totality, you may be having millions of records you 
don’t want the all the records collected on the driver. You would like to save it 
to the file, but don’t call collect, it leads out of memory on driver. It is always preferred like sample and take 
->>> Avoid unnecessary actions. Each action triggers one spark job, we can cache 
the RDD if multiple actions are needed. We can cache or repartition a bigger RDD 
if it has to be constantly joined with smaller datasets. 
If we have resources at your disposal, you can increase the availability 
of each partition by replicating it, you can specify underscore2, 
in your storage level like memory only, underscore2 memory and disk and so on. 
Even if one machine goes down the cache data is present on some other machine 
->>> If we have to collect metrics, metrics which can be collected via accumulators can avoid additional actions, 
->>> If we need to check if an RDD is empty or not
->>> Instead of using if rdd.count() > 0
->>> Use, if len(rdd.take(1)) > 0
->>> Count will scan entire RDD, whereas take() will just read only those partitions from where results can be retrieved

## the repartition algorith does a full shuffle and creates new partitions with data that's distributed evenly. (stack Overflow)
## Coalesce avoids a full shuffle. 
If it is known that the number is decreasing then the executor can safely keep data on the minimum number of partitions, 
only moving the data off the extra nodes, onto the nodes that we kept.(stackoverflow)
## Coalesce uses existing partitions to minimize the amount of data that's shuffled. Repartition creates new partitions 
and does a full shuffle. Coalesce results in partitions with different amounts of data 9sometimes partitions 
that have much different sizes) and repartition results in roughly equal sized partitions.(Stackoverflow)
## coalesce may run faster than repartition, but unequal sized partitions are generally slower to work with than equal
sized partitions. You will usually need to repartition datasets after filtering a large data set.
He mentioned extra more that, spark is build for to work for equal sized partitions. 9stackoverflow)
# repartition will ignore existing partitions and create new ones. So you can use it to fix data skew. 
You can mention partition keys to define the distribution. Data skew is one of the biggest problems in the
'big data' problem space. Other thing is in repartition by providing the number of partitions,
it tries to redistribute the data uniformly on all the partitions while in case of Coalesce 
we could still have skew data in some cases.(stackoverflow)


In [0]:
df = spark.read.csv("dbfs:/FileStore/shared_uploads/sudippandit99@gmail.com/finance.xlsx", header =True)

In [0]:
df.rdd.getNumPartitions()

Out[12]: 1

In [0]:
df1 = df.repartition(5)

In [0]:
df1.count()

Out[16]: 591

In [0]:
df2 = df.coalesce(3)

In [0]:
df2.count()

Out[19]: 591